In [184]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text")
    return text

In [298]:
import pdfplumber
def iou(bbox1, bbox2, left=False):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.
    bbox1 and bbox2 are tuples of (x0, y0, x1, y1).
    """
    x0 = max(bbox1[0], bbox2[0])
    y0 = max(bbox1[1], bbox2[1])
    x1 = min(bbox1[2], bbox2[2])
    y1 = min(bbox1[3], bbox2[3])

    intersection_area = max(0, x1 - x0) * max(0, y1 - y0)
    bbox1_area = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
    bbox2_area = (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])

    union_area = bbox1_area + bbox2_area - intersection_area if not left else bbox1_area
    return intersection_area / union_area if union_area > 0 else 0

def detect_paragraphs(pdf_path):
    paragraphs = []
    all_tables = []
    with pdfplumber.open(pdf_path) as pdf:
        print(len(pdf.pages))
        for page_number, page in enumerate(pdf.pages):
            # Extract text lines with their bounding boxes
            tables = page.find_tables()
            table_areas = []
            words = page.extract_words(x_tolerance= 1., 
                                        y_tolerance=1.35,
                                        keep_blank_chars=True, 
                                        use_text_flow=True)
            # If the page has tables, we can extract text from them
            for table_index, table in enumerate(tables):
                table_bbox = table.bbox
                table_areas.append(table_bbox)
                table_data = []
                for row in table.rows: 
                    row_data = []
                    print(row.cells)
                    for cell in row.cells:
                        print(cell)
                        if cell is None:
                            row_data.append("")
                            continue
                        cell_words = [w for w in words if iou([w['x0'], w['top'], w['x1'], w['bottom']], cell, left=True) >= 0.9]
                        cell_text = " ".join([w['text'] for w in cell_words])
                        row_data.append(cell_text)
                    table_data.append(row_data)
                all_tables.append(table_data)
            
            current_paragraph_words = []
            for i, word in enumerate(words):
                valid = True
                for bbox in table_areas:
                    word_bbox = [word['x0'], word['top'], word['x1'], word['bottom']]
                    # Check if the line intersects with any table area
                    if (iou(word_bbox, bbox, left=True) > 0.8):
                        # If it does, we skip adding this line to paragraphs
                        valid = False
                        break
                    else:
                        print(line['text'], word_bbox, bbox, iou(word_bbox, bbox, left=True))
                if not valid:
                    print(f"Skipping line due to table intersection: {word['text']}")
                    continue
                current_paragraph_words.append(word['text'])
                # Heuristic: Check for a significant vertical jump or end of page
                if i + 1 < len(words):
                    next_word = words[i+1]
                    # Adjust threshold based on typical line spacing in your PDFs
                    if next_word['top'] - word['bottom'] > 0.5 * word['height']:
                        paragraphs.append(" ".join(current_paragraph_words))
                        current_paragraph_words = []
                else: # End of page
                    paragraphs.append(" ".join(current_paragraph_words))

    return paragraphs, all_tables # Print the first 500 characters of the extracted text

In [299]:
pdf_text, tables = detect_paragraphs(r"C:\ecoit_summer\data\Mẫu phê duyệt tờ trình của KT (đơn giản)\03. Báo cáo thẩm tra.pdf")

3


In [300]:
for p in pdf_text:   
    print(p.replace(";", "\n").replace("  ", " ")) 

TỔNG CÔNG TY  ĐIỆN LỰC TP HÀ NỘI BAN TỔNG HỢP  CỘNG HOÀ XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc              Hà Nội, ngày 24 tháng 01 năm 2025  
BÁO CÁO THẨM TRA Về việc thông qua chủ trương ban hành Tiêu chuẩn kỹ thuật thiết bị bay không người lái (UAV) trong Tổng công ty Điện lực TP Hà Nội  Kính gửi: Hội đồng Thành viên Tổng Công ty Điện lực TP Hà Nội.   Căn cứ Quyết định số 401/QĐ-EVN ngày 14/10/2019 của Hội đồng thành viên Tập đoàn Điện lực Việt Nam về việc phê duyệt Điều lệ Tổ chức và hoạt động của Tổng công ty Điện lực TP Hà Nội
 Quyết định số 118/QĐ- HĐTV ngày 05/8/2022 của Hội đồng thành viên Tập đoàn Điện lực Việt Nam về việc sửa đổi, bổ sung Điều lệ tổ chức và hoạt động của Tổng công ty Điện lực TP Hà Nội
  Căn cứ Quyết định số 123/QĐ-HĐTV ngày 01/10/2021 của Hội đồng thành viên Tập đoàn Điện lực Việt Nam về việc ban hành Quy chế Quản trị trong Tập đoàn Điện lực Quốc gia Việt Nam
  Căn cứ Quyết định số 03/QĐ-HĐTV ngày 01/10/2019 của Hội đồng thành viên Tổng công

IndexError: list index out of range

In [226]:
import copy
executed = ['', '', '']
res_table = []
for row in tables[0]:
    row_data = [row[1], row[4], row[10]]
    row_data = ['' + data for data in row_data]
    if row_data[0] != '':
        if executed[0] != '' and row_data[0] != executed[0]:
            res_table.append(executed)
            executed = ['', '', '']

        if '.' not in row_data[0]:
            res_table.append(row_data)
            # executed = row_data
            continue
    
    for index, data in enumerate(row_data):
        if executed[index] != '' and data != '':
            executed[index] += ' '
        executed[index] += data

In [ ]:
import pdfplumber

pdf_path = r"C:\ecoit_summer\data\Mẫu phê duyệt tờ trình của KT (đơn giản)\03. 1phiếu trình.pdf"

full_text = ""
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        # Extract text with layout (preserve whitespace)
        page_text = page.extract_text(x_tolerance=1, y_tolerance=1, layout=True)
        if page_text:
            full_text += page_text + "\n\n"

print(full_text)